In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '282 fBodyAcc-energy()-X',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 40),
            classifier_block(40, 30),
            classifier_block(30, 25),
            classifier_block(25, 25),
            nn.Linear(25, 27)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

# Train on Real Test on Real

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_10 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_11 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_12 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_13 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_14 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_15 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_16 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_17 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_18 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_19 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_20 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_21 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_22 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_23 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_24 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_25 = GAN_data[(GAN_data['Subject'] == 19) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_26 = GAN_data[(GAN_data['Subject'] == 19) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_27 = GAN_data[(GAN_data['Subject'] == 19) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9, X_10, X_11, X_12, X_13, X_14, X_15, X_16, X_17, X_18, X_19, X_20, X_21, X_22, X_23, X_24, X_25, X_26, X_27))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9) + [9] * len(X_10) + [10] * len(X_11) + [11] * len(X_12) + [12] * len(X_13) + [13] * len(X_14) + [14] * len(X_15) + [15] * len(X_16) + [16] * len(X_17) + [17] * len(X_18) + [18] * len(X_19) + [19] * len(X_20) + [20] * len(X_21) + [21] * len(X_22) + [22] * len(X_23) + [23] * len(X_24) + [24] * len(X_25) + [25] * len(X_26) + [26] * len(X_27)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11, 14, 17, 19]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [8]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 16.505478858947754, Final Batch Loss: 3.3067429065704346
Epoch 2, Loss: 16.490118741989136, Final Batch Loss: 3.297823429107666
Epoch 3, Loss: 16.476760149002075, Final Batch Loss: 3.2878243923187256
Epoch 4, Loss: 16.466208696365356, Final Batch Loss: 3.298814535140991
Epoch 5, Loss: 16.44110941886902, Final Batch Loss: 3.2898590564727783
Epoch 6, Loss: 16.40273141860962, Final Batch Loss: 3.277898073196411
Epoch 7, Loss: 16.35971736907959, Final Batch Loss: 3.2479093074798584
Epoch 8, Loss: 16.297749519348145, Final Batch Loss: 3.26343035697937
Epoch 9, Loss: 16.18058490753174, Final Batch Loss: 3.2468974590301514
Epoch 10, Loss: 16.045623064041138, Final Batch Loss: 3.22884202003479
Epoch 11, Loss: 15.78481912612915, Final Batch Loss: 3.1396026611328125
Epoch 12, Loss: 15.400501489639282, Final Batch Loss: 2.9925537109375
Epoch 13, Loss: 14.991413831710815, Final Batch Loss: 2.951744318008423
Epoch 14, Loss: 14.63778305053711, Final Batch Loss: 2.922180652618408
Epoch

Epoch 118, Loss: 6.6184492111206055, Final Batch Loss: 1.320454716682434
Epoch 119, Loss: 6.455546259880066, Final Batch Loss: 1.3355871438980103
Epoch 120, Loss: 6.665382623672485, Final Batch Loss: 1.277972936630249
Epoch 121, Loss: 6.559099078178406, Final Batch Loss: 1.1676918268203735
Epoch 122, Loss: 6.641663074493408, Final Batch Loss: 1.3091872930526733
Epoch 123, Loss: 6.455274820327759, Final Batch Loss: 1.2672171592712402
Epoch 124, Loss: 6.540860176086426, Final Batch Loss: 1.3517279624938965
Epoch 125, Loss: 6.40455961227417, Final Batch Loss: 1.3054262399673462
Epoch 126, Loss: 6.352289915084839, Final Batch Loss: 1.2171385288238525
Epoch 127, Loss: 6.529070734977722, Final Batch Loss: 1.426863670349121
Epoch 128, Loss: 6.424088001251221, Final Batch Loss: 1.3234577178955078
Epoch 129, Loss: 6.313281297683716, Final Batch Loss: 1.2204432487487793
Epoch 130, Loss: 6.413975715637207, Final Batch Loss: 1.2240418195724487
Epoch 131, Loss: 6.374463438987732, Final Batch Loss: 

Epoch 232, Loss: 5.6061296463012695, Final Batch Loss: 1.0679527521133423
Epoch 233, Loss: 5.563033699989319, Final Batch Loss: 1.0629878044128418
Epoch 234, Loss: 5.616850972175598, Final Batch Loss: 1.2043125629425049
Epoch 235, Loss: 5.302647829055786, Final Batch Loss: 0.9159921407699585
Epoch 236, Loss: 5.566902160644531, Final Batch Loss: 1.2495670318603516
Epoch 237, Loss: 5.645352721214294, Final Batch Loss: 1.0977498292922974
Epoch 238, Loss: 5.658755660057068, Final Batch Loss: 1.3949236869812012
Epoch 239, Loss: 5.570740699768066, Final Batch Loss: 1.2214813232421875
Epoch 240, Loss: 5.613604187965393, Final Batch Loss: 1.081877589225769
Epoch 241, Loss: 5.463521599769592, Final Batch Loss: 1.0589697360992432
Epoch 242, Loss: 5.621071696281433, Final Batch Loss: 1.1515741348266602
Epoch 243, Loss: 5.578575611114502, Final Batch Loss: 1.0596535205841064
Epoch 244, Loss: 5.38246214389801, Final Batch Loss: 0.9752969741821289
Epoch 245, Loss: 5.584788799285889, Final Batch Loss

Epoch 350, Loss: 4.914102673530579, Final Batch Loss: 0.9604529142379761
Epoch 351, Loss: 5.037102699279785, Final Batch Loss: 1.00411057472229
Epoch 352, Loss: 5.12628835439682, Final Batch Loss: 1.0726748704910278
Epoch 353, Loss: 4.963432192802429, Final Batch Loss: 0.9592660665512085
Epoch 354, Loss: 4.985727369785309, Final Batch Loss: 1.0726711750030518
Epoch 355, Loss: 5.079389929771423, Final Batch Loss: 1.0405670404434204
Epoch 356, Loss: 5.060293555259705, Final Batch Loss: 1.0306192636489868
Epoch 357, Loss: 5.0418906807899475, Final Batch Loss: 1.0090638399124146
Epoch 358, Loss: 4.923640191555023, Final Batch Loss: 0.9607962965965271
Epoch 359, Loss: 4.890185594558716, Final Batch Loss: 0.9858558177947998
Epoch 360, Loss: 4.988581001758575, Final Batch Loss: 0.8825023174285889
Epoch 361, Loss: 4.9076040387153625, Final Batch Loss: 1.0162402391433716
Epoch 362, Loss: 4.859933018684387, Final Batch Loss: 0.8804076313972473
Epoch 363, Loss: 4.797532856464386, Final Batch Loss

Epoch 464, Loss: 4.522627949714661, Final Batch Loss: 0.916621744632721
Epoch 465, Loss: 4.426447153091431, Final Batch Loss: 0.915871798992157
Epoch 466, Loss: 4.519600570201874, Final Batch Loss: 0.8338197469711304
Epoch 467, Loss: 4.597135782241821, Final Batch Loss: 0.9392505288124084
Epoch 468, Loss: 4.377130568027496, Final Batch Loss: 0.8347110152244568
Epoch 469, Loss: 4.591818869113922, Final Batch Loss: 0.8208867311477661
Epoch 470, Loss: 4.39230340719223, Final Batch Loss: 0.9611878395080566
Epoch 471, Loss: 4.447760343551636, Final Batch Loss: 0.7779467105865479
Epoch 472, Loss: 4.621935725212097, Final Batch Loss: 1.0190246105194092
Epoch 473, Loss: 4.50722998380661, Final Batch Loss: 0.8831626176834106
Epoch 474, Loss: 4.489015758037567, Final Batch Loss: 0.897365391254425
Epoch 475, Loss: 4.43985915184021, Final Batch Loss: 0.8610922694206238
Epoch 476, Loss: 4.43778383731842, Final Batch Loss: 0.9276560544967651
Epoch 477, Loss: 4.528825461864471, Final Batch Loss: 0.88

...

Epoch 4482, Loss: 1.3794673383235931, Final Batch Loss: 0.3149467706680298
Epoch 4483, Loss: 1.3716305196285248, Final Batch Loss: 0.27675342559814453
Epoch 4484, Loss: 1.3868289142847061, Final Batch Loss: 0.35422953963279724
Epoch 4485, Loss: 1.4639737010002136, Final Batch Loss: 0.2798301875591278
Epoch 4486, Loss: 1.451229840517044, Final Batch Loss: 0.24154701828956604
Epoch 4487, Loss: 1.528078943490982, Final Batch Loss: 0.35599473118782043
Epoch 4488, Loss: 1.4712002575397491, Final Batch Loss: 0.29378050565719604
Epoch 4489, Loss: 1.5642710030078888, Final Batch Loss: 0.3161327838897705
Epoch 4490, Loss: 1.4218148440122604, Final Batch Loss: 0.35875922441482544
Epoch 4491, Loss: 1.3865301460027695, Final Batch Loss: 0.24814210832118988
Epoch 4492, Loss: 1.7215997576713562, Final Batch Loss: 0.4326619505882263
Epoch 4493, Loss: 1.356339082121849, Final Batch Loss: 0.31319284439086914
Epoch 4494, Loss: 1.2962770611047745, Final Batch Loss: 0.2444286197423935
Epoch 4495, Loss: 1.

Epoch 4595, Loss: 1.3693108409643173, Final Batch Loss: 0.3454481065273285
Epoch 4596, Loss: 1.7200629711151123, Final Batch Loss: 0.42610201239585876
Epoch 4597, Loss: 1.4026943296194077, Final Batch Loss: 0.29689493775367737
Epoch 4598, Loss: 1.4112889617681503, Final Batch Loss: 0.4119621515274048
Epoch 4599, Loss: 1.5359580218791962, Final Batch Loss: 0.32374539971351624
Epoch 4600, Loss: 1.3687408864498138, Final Batch Loss: 0.28218674659729004
Epoch 4601, Loss: 1.3948226422071457, Final Batch Loss: 0.3152041435241699
Epoch 4602, Loss: 1.3462525457143784, Final Batch Loss: 0.2094365954399109
Epoch 4603, Loss: 1.3816570192575455, Final Batch Loss: 0.3201456367969513
Epoch 4604, Loss: 1.2497546821832657, Final Batch Loss: 0.1628057360649109
Epoch 4605, Loss: 1.4559652507305145, Final Batch Loss: 0.30186009407043457
Epoch 4606, Loss: 1.4514989256858826, Final Batch Loss: 0.2840862274169922
Epoch 4607, Loss: 1.3703297823667526, Final Batch Loss: 0.2686794698238373
Epoch 4608, Loss: 1.

Epoch 4704, Loss: 1.40388223528862, Final Batch Loss: 0.3343307375907898
Epoch 4705, Loss: 1.5142685174942017, Final Batch Loss: 0.23805150389671326
Epoch 4706, Loss: 1.3401524722576141, Final Batch Loss: 0.31655600666999817
Epoch 4707, Loss: 1.502029538154602, Final Batch Loss: 0.3724460303783417
Epoch 4708, Loss: 1.412722334265709, Final Batch Loss: 0.2869187593460083
Epoch 4709, Loss: 1.454043760895729, Final Batch Loss: 0.24469523131847382
Epoch 4710, Loss: 1.3200305104255676, Final Batch Loss: 0.21288418769836426
Epoch 4711, Loss: 1.3619694113731384, Final Batch Loss: 0.23574811220169067
Epoch 4712, Loss: 1.3799008429050446, Final Batch Loss: 0.23866485059261322
Epoch 4713, Loss: 1.6278861463069916, Final Batch Loss: 0.4625062644481659
Epoch 4714, Loss: 1.3892932683229446, Final Batch Loss: 0.3509638011455536
Epoch 4715, Loss: 1.488582968711853, Final Batch Loss: 0.2612456679344177
Epoch 4716, Loss: 1.4429599344730377, Final Batch Loss: 0.2559073865413666
Epoch 4717, Loss: 1.33087

Epoch 4817, Loss: 1.3726776838302612, Final Batch Loss: 0.24008594453334808
Epoch 4818, Loss: 1.3359842002391815, Final Batch Loss: 0.23973402380943298
Epoch 4819, Loss: 1.2302511185407639, Final Batch Loss: 0.24165058135986328
Epoch 4820, Loss: 1.360974445939064, Final Batch Loss: 0.20118875801563263
Epoch 4821, Loss: 1.4147828370332718, Final Batch Loss: 0.3095092475414276
Epoch 4822, Loss: 1.6417720019817352, Final Batch Loss: 0.3985663950443268
Epoch 4823, Loss: 1.378714844584465, Final Batch Loss: 0.27037209272384644
Epoch 4824, Loss: 1.4727988690137863, Final Batch Loss: 0.39866727590560913
Epoch 4825, Loss: 1.2987442910671234, Final Batch Loss: 0.25449514389038086
Epoch 4826, Loss: 1.3864396661520004, Final Batch Loss: 0.22316084802150726
Epoch 4827, Loss: 1.4454704821109772, Final Batch Loss: 0.25118353962898254
Epoch 4828, Loss: 1.2221512645483017, Final Batch Loss: 0.2405805140733719
Epoch 4829, Loss: 1.4121105074882507, Final Batch Loss: 0.4148446321487427
Epoch 4830, Loss: 

Epoch 4932, Loss: 1.2748288810253143, Final Batch Loss: 0.3089393079280853
Epoch 4933, Loss: 1.404331773519516, Final Batch Loss: 0.3732761740684509
Epoch 4934, Loss: 1.2747384011745453, Final Batch Loss: 0.2681877017021179
Epoch 4935, Loss: 1.3502983748912811, Final Batch Loss: 0.32182154059410095
Epoch 4936, Loss: 1.4673156440258026, Final Batch Loss: 0.3570214509963989
Epoch 4937, Loss: 1.2973094582557678, Final Batch Loss: 0.26664671301841736
Epoch 4938, Loss: 1.4843515753746033, Final Batch Loss: 0.31237560510635376
Epoch 4939, Loss: 1.5379494577646255, Final Batch Loss: 0.4421100616455078
Epoch 4940, Loss: 1.381883680820465, Final Batch Loss: 0.25995883345603943
Epoch 4941, Loss: 1.420852705836296, Final Batch Loss: 0.22433942556381226
Epoch 4942, Loss: 1.172381803393364, Final Batch Loss: 0.26922938227653503
Epoch 4943, Loss: 1.3209745287895203, Final Batch Loss: 0.24417488276958466
Epoch 4944, Loss: 1.296840250492096, Final Batch Loss: 0.25074347853660583
Epoch 4945, Loss: 1.30

In [9]:
softmax = nn.Softmax(dim = 1)
model.train()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[23  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   1  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  1]
 [ 0  0  0 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  2  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  1
   0  0  0]
 [ 0  0  0  0  0  0  9  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  5  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 13  0  0  0  0  0  

# Train on Fake Test on Real

In [10]:
# 20,000 epochs DEFAULT PARAMETERS FOR THRESHOLDS

In [11]:
gen = Generator(z_dim = 112)
load_model(gen, "3 Label 9 Subject GAN Ablation_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=112, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=30, bias=True)
    (4): Tanh()
  )
)

In [25]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 9)

fake_labels = []

for k in range(size):
    if act_vectors[0][k] == 0 and usr_vectors[0][k] == 0:
        fake_labels.append(0)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 0:
        fake_labels.append(1)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 0:
        fake_labels.append(2)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 1:
        fake_labels.append(3)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 1:
        fake_labels.append(4)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 1:
        fake_labels.append(5)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 2:
        fake_labels.append(6)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 2:
        fake_labels.append(7)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 2:
        fake_labels.append(8)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 3:
        fake_labels.append(9)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 3:
        fake_labels.append(10)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 3:
        fake_labels.append(11)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 4:
        fake_labels.append(12)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 4:
        fake_labels.append(13)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 4:
        fake_labels.append(14)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 5:
        fake_labels.append(15)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 5:
        fake_labels.append(16)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 5:
        fake_labels.append(17)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 6:
        fake_labels.append(18)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 6:
        fake_labels.append(19)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 6:
        fake_labels.append(20)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 7:
        fake_labels.append(21)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 7:
        fake_labels.append(22)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 7:
        fake_labels.append(23)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 8:
        fake_labels.append(24)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 8:
        fake_labels.append(25)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 8:
        fake_labels.append(26)
        
fake_labels = np.asarray(fake_labels)
to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()
#fake_features = gen(to_gen).detach().numpy()

In [26]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5, zero_division = 0))

[[ 9  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  1  0  0
   0  0  0]
 [ 0  0  0  0 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  1  0  0 12  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
   0  0  0]
 [ 0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  7  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  4  0  3  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  